### DK Salaries Optimizer

In [1]:
import pandas as pd
import numpy as np
from pulp import *

In [2]:
dk_data = pd.read_csv("data/dksalaries_nhl.csv")

# Get the fields that are useful
dk_data = dk_data[['Name', 'Roster Position', 'Salary', 'TeamAbbrev', 'AvgPointsPerGame']]

dk_data.head()

,Name,Roster Position,Salary,TeamAbbrev,AvgPointsPerGame
0,Robin Lehner,G,8100,VGK,17.09
1,Marc-Andre Fleury,G,7900,VGK,12.64
2,Carter Hart,G,7800,PHI,13.26
3,Semyon Varlamov,G,7700,NYI,13.49
4,Brian Elliott,G,7600,PHI,10.92


In [49]:
injured_players = ['Jacob Markstrom', 'Oskar Lindblom']

In [50]:
dk_data = dk_data[~dk_data['Name'].isin(injured_players)]

In [51]:
# Roster Positions for NBA 
nhl_roster_positions = ['C', 'W', 'D', 'G', 'UTIL']

SALARY_CAP = 50000
MAX_PLAYERS = 9


def make_vars(name, position, roster_position):
    if position in roster_position:
        return LpVariable(name + '_' + position,lowBound=0, cat='Binary')
    return 0

for position in nhl_roster_positions:
    dk_data[position] = dk_data.apply(lambda x: make_vars(x['Name'],position, x['Roster Position']),axis=1)

    #[1 if position in x else 0 for x in dk_data['Roster Position']]

In [52]:
prob = LpProblem("OptimizeDK",LpMaximize)

In [53]:
names = dk_data['Name']
salaries = dk_data['Salary']
projected_points = dk_data['AvgPointsPerGame']

In [54]:
dk_data.head()

,Name,Roster Position,Salary,TeamAbbrev,AvgPointsPerGame,C,W,D,G,UTIL
0,Robin Lehner,G,8100,VGK,17.09,0,0,0,Robin_Lehner_G,0
1,Marc-Andre Fleury,G,7900,VGK,12.64,0,0,0,Marc_Andre_Fleury_G,0
2,Carter Hart,G,7800,PHI,13.26,0,0,0,Carter_Hart_G,0
3,Semyon Varlamov,G,7700,NYI,13.49,0,0,0,Semyon_Varlamov_G,0
4,Brian Elliott,G,7600,PHI,10.92,0,0,0,Brian_Elliott_G,0


In [55]:
# The objective function is added to 'prob' first
# OBJ FUNCTION
prob += lpSum([dk_data[i] * projected_points for i in nhl_roster_positions]), 'Projected Points'

In [56]:
# CONSTRAINTS

# Stay under salary cap
prob += lpSum(salaries * dk_data[i] for i in nhl_roster_positions) <= SALARY_CAP

# Stay under Num Players
prob += lpSum(dk_data[i] for i in nhl_roster_positions) <= MAX_PLAYERS

# Select one of each position
prob += lpSum([dk_data['C']]) >= 2
prob += lpSum([dk_data['W']]) >= 3
prob += lpSum([dk_data['D']]) >= 2
prob += lpSum([dk_data['G']]) >= 1
prob += lpSum([dk_data['UTIL']]) >= 1


# Don't use same player twice
for index, row in dk_data.iterrows():
    prob += lpSum(row[i] for i in nhl_roster_positions) <= 1


In [57]:
# prob

In [58]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [59]:
prob.status

1

In [45]:
print("Total Estimated Points = ", value(prob.objective))


Total Estimated Points =  104.79


In [46]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    if v.varValue == 1:
        print(v.name)
        v.getName

Alexander_Edler_D
Bo_Horvat_C
Brock_Boeser_UTIL
J.T._Miller_W
Max_Pacioretty_W
Patrick_Brown_C
Robin_Lehner_G
Ryan_Pulock_D
Tanner_Pearson_W
